In [1]:
import sys
import os
import glob
home_folder = os.path.dirname(os.getcwd())
sys.path.append(home_folder)
os.environ['MALLET_HOME'] = '/mnt/packages/Mallet/bin/mallet'

import json
from src.xml_server_connect import *
from src.utility import *
from src.extract_table import *
from src.pre_process import *
from src.lda_model import *
from collections import defaultdict
import pandas as pd
import numpy as np
import re
import ftfy
import string
import unicodedata
import calendar
from multiprocessing import Pool
import nltk
import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim import corpora
import datetime
import pyLDAvis
import pyLDAvis.gensim as gensim_vis
from gensim.models import CoherenceModel

DEBUG:$HOME=/home/ubuntu
DEBUG:matplotlib data path /usr/local/anaconda/lib/python3.6/site-packages/matplotlib/mpl-data
DEBUG:loaded rc file /home/ubuntu/.config/matplotlib/matplotlibrc
DEBUG:matplotlib version 2.2.2
DEBUG:interactive is False
DEBUG:platform is linux
DEBUG:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', '_bootlocale', '_locale', 'types', 'functools', '_functools', 'collections', 'operator', '_operator', 'keyword', 'heapq', '_heapq', 'itertools', 'reprlib', '_collections', 'weakref', 'collections.abc', 'importlib', 'importlib._bootstrap'

INFO:'pattern' package not found; tag filters are not available for English


In [2]:
input_folder = '/mnt/inputs'
dataset_folder = '/domino/dataset/raw_report'
dataset_processed_folder = '/domino/dataset/processed'
if not os.path.exists(dataset_processed_folder):
    dataset_processed_folder = '/domino/dataset/output'
lemma_text_file = 'RAM_lemma.json'
lda_dictionary_file = 'word_dictionary.txt'
corpus_file = 'RAM_lda_corpus.mm'
mallet_path = '/mnt/packages/Mallet/bin/mallet'
model_folder = '/mnt/models'
result_folder = '/mnt/results'
tmp_folder = '/tmp'

In [3]:
def save_results_to_excel(date, model, model_dict, lemma_folder, raw_text_folder, n_topics, n_words, id_map, result_folder):
    df_topic_word = pd.DataFrame(data = np.zeros((n_topics, n_words)), columns= ['word'+ str(x) for x in range(n_words)])
    for i in range(n_topics):
        df_topic_word.iloc[i] = pd.DataFrame(model.show_topic(topicid= i, topn= n_words))[0].tolist()
    df_topic_word['viz_topic_id'] = [id_map[key] for key in df_topic_word.index]
    df_topic_word.set_index('viz_topic_id', inplace=True)
    
    doc_records = []
    report_files_list = glob.glob(lemma_folder+'/*.json')
    for report_file in report_files_list:
        with open(report_file, 'r', encoding='utf8') as f:
            data = json.load(f)
            lemma_content = data['lemma_content']
            country = data['country']
            year = data['year']
        with open(os.path.join(raw_text_folder, '{0}_{1}.json'.format(country.replace(' ','_').replace('-','_'),str(year))), 'r', encoding='utf8') as f:
            raw_xml = json.load(f)['xml']
        raw_text = extract_table_content(extract_risk_table_node_from_report(raw_xml))
        
        doc_record = {}
        doc_record['year'] = year
        doc_record['country'] = country
        doc_record['text'] = raw_text
        doc_topic = model.get_document_topics(model_dict.doc2bow(lemma_content))
        for i in range(n_topics):
            doc_record['topic_%d'%id_map[i]] = 0
        for i, v in doc_topic:
            doc_record['topic_%d'%id_map[i]] = v
        doc_records.append(doc_record)
    df_doc_records = pd.DataFrame.from_records(doc_records)
    writer = pd.ExcelWriter(path = os.path.join(result_folder,'Mallet_{}_topics_{}.xlsx'.format(n_topics, date)))
    df_topic_word.to_excel(writer, 'Toipc and Key Word')
    df_doc_records.to_excel(writer, 'Document and Topic')
    return df_doc_records, df_topic_word

<h4>Read and Process Reports to LDA Corpus [list of (int,int)] and Save to inputs folder <br> (re-run all the dictionary and processed reports generation) <h4>

In [17]:
xml_report_to_lemma_doc(skip_set=set(['CurrencyName']))

Have processed 262 number of reports (out of 1925 reports)
Have processed 484 number of reports (out of 1925 reports)
Have processed 729 number of reports (out of 1925 reports)
Have processed 921 number of reports (out of 1925 reports)
Have processed 1151 number of reports (out of 1925 reports)
Have processed 1415 number of reports (out of 1925 reports)
Have processed 1689 number of reports (out of 1925 reports)


In [4]:
#lda_dict = build_dictionary(xml_report_lemma_gen(dataset_folder))
lda_dict = build_dictionary_from_files(dataset_processed_folder)
save_dictionary(os.path.join(input_folder, lda_dictionary_file), lda_dict)

lemma_corpus = []
processed_files_list = glob.glob(dataset_processed_folder + '/*.json')
total_no = len(processed_files_list)
count = 0
start = time.time()
for processed_file in processed_files_list:
    count += 1
    with open(processed_file, 'r', encoding='utf8') as f:
        lemma_content = json.load(f)['lemma_content']
    
    lemma_corpus.append(lemma_content)
    if (time.time()-start)/60 > 5:
        print('Have processed %d number of reports (out of %d reports)'%(count, len(total_no)))
        start = time.time()
              
data = {'lemma_text':lemma_corpus}
with open(os.path.join(input_folder, lemma_text_file), 'w', encoding='utf8') as f:
    json.dump(data, f)
    
corpus_gen = lda_corpus_gen_from_files(dataset_processed_folder, lda_dict)
save_corpus(corpus_gen, os.path.join(input_folder, corpus_file))

INFO:adding document #0 to Dictionary(0 unique tokens: [])
INFO:built Dictionary(239 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 1 documents (total 357 corpus positions)
INFO:adding document #0 to Dictionary(239 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(366 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 2 documents (total 583 corpus positions)
INFO:adding document #0 to Dictionary(366 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(486 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 3 documents (total 829 corpus positions)
INFO:adding document #0 to Dictionary(486 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(591 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)

INFO:built Dictionary(1876 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 30 documents (total 8670 corpus positions)
INFO:adding document #0 to Dictionary(1876 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(1922 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 31 documents (total 9198 corpus positions)
INFO:adding document #0 to Dictionary(1922 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(1934 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 32 documents (total 9428 corpus positions)
INFO:adding document #0 to Dictionary(1934 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(1959 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 33 documents (total 9797 corpus position

INFO:built Dictionary(2424 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 59 documents (total 17646 corpus positions)
INFO:adding document #0 to Dictionary(2424 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(2433 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 60 documents (total 17871 corpus positions)
INFO:adding document #0 to Dictionary(2433 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(2437 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 61 documents (total 17993 corpus positions)
INFO:adding document #0 to Dictionary(2437 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(2458 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 62 documents (total 18291 corpus posi

INFO:built Dictionary(2843 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 88 documents (total 25650 corpus positions)
INFO:adding document #0 to Dictionary(2843 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(2848 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 89 documents (total 25846 corpus positions)
INFO:adding document #0 to Dictionary(2848 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(2856 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 90 documents (total 26172 corpus positions)
INFO:adding document #0 to Dictionary(2856 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(2863 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 91 documents (total 26393 corpus posi

INFO:built Dictionary(3145 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 117 documents (total 33079 corpus positions)
INFO:adding document #0 to Dictionary(3145 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(3169 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 118 documents (total 33414 corpus positions)
INFO:adding document #0 to Dictionary(3169 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(3178 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 119 documents (total 33656 corpus positions)
INFO:adding document #0 to Dictionary(3178 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(3209 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 120 documents (total 34152 corpus 

...

INFO:built Dictionary(5210 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 581 documents (total 169862 corpus positions)
INFO:adding document #0 to Dictionary(5210 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(5213 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 582 documents (total 170351 corpus positions)
INFO:adding document #0 to Dictionary(5213 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(5216 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 583 documents (total 170729 corpus positions)
INFO:adding document #0 to Dictionary(5216 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(5219 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 584 documents (total 171052 cor

INFO:built Dictionary(5266 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 610 documents (total 179214 corpus positions)
INFO:adding document #0 to Dictionary(5266 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(5266 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 611 documents (total 179679 corpus positions)
INFO:adding document #0 to Dictionary(5266 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(5269 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 612 documents (total 179966 corpus positions)
INFO:adding document #0 to Dictionary(5269 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(5269 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 613 documents (total 180134 cor

INFO:built Dictionary(5327 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 639 documents (total 188618 corpus positions)
INFO:adding document #0 to Dictionary(5327 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(5327 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 640 documents (total 188760 corpus positions)
INFO:adding document #0 to Dictionary(5327 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(5333 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 641 documents (total 189194 corpus positions)
INFO:adding document #0 to Dictionary(5333 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(5339 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 642 documents (total 189434 cor

INFO:built Dictionary(5395 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 668 documents (total 196543 corpus positions)
INFO:adding document #0 to Dictionary(5395 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...)
INFO:built Dictionary(5395 unique tokens: ['abenomics', 'accelerate', 'achieve', 'activity', 'addition']...) from 669 documents (total 197015 corpus positions)
INFO:saving dictionary mapping to /mnt/inputs/word_dictionary.txt
INFO:storing corpus in Matrix Market format to /mnt/inputs/RAM_lda_corpus.mm
INFO:saving sparse matrix to /mnt/inputs/RAM_lda_corpus.mm
INFO:PROGRESS: saving document #0
INFO:saved 669x5395 matrix, density=3.480% (125591/3609255)
INFO:saving MmCorpus index to /mnt/inputs/RAM_lda_corpus.mm.index


It took 0.07 min to process report and build lda corpus


In [12]:
load_dict_indicator = False

if load_dict_indicator:
    lda_dict = load_dictionary(os.path.join(input_folder, lda_dictionary_file))

#corpus_gen = lda_corpus_gen(xml_report_lemma_gen(dataset_folder), lda_dict)
corpus_gen = lda_corpus_gen_from_files(dataset_processed_folder, lda_dict)
save_corpus(corpus_gen, os.path.join(input_folder, corpus_file))

INFO:storing corpus in Matrix Market format to /mnt/inputs/RAM_lda_corpus.mm
INFO:saving sparse matrix to /mnt/inputs/RAM_lda_corpus.mm
INFO:PROGRESS: saving document #0
INFO:saved 669x5395 matrix, density=3.480% (125591/3609255)
INFO:saving MmCorpus index to /mnt/inputs/RAM_lda_corpus.mm.index


It took 0.07 min to process report and build lda corpus


<h4>Load LDA Dictionary and Processed Corpus<h4>

In [11]:
with open(os.path.join(input_folder, lemma_text_file), 'r', encoding='utf8') as f:
    data = json.load(f)
    lemma_corpus = data['lemma_text']
    
lda_dict = load_dictionary(os.path.join(input_folder, lda_dictionary_file))
RAM_corpus = load_corpus(os.path.join(input_folder, corpus_file))

INFO:loaded corpus index from /mnt/inputs/RAM_lda_corpus.mm.index
INFO:initializing cython corpus reader from /mnt/inputs/RAM_lda_corpus.mm
INFO:accepted corpus with 669 documents, 5395 features, 125591 non-zero entries


<h4>Train and Tuning Hyperparameters<h4>

In [23]:
def fine_tune_lda_topics(corpus, dictionary, texts, mallet_path, start=10, end=50, step=1, alpha=1):
    """
    Compute c_v coherence for various number of topics
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    end : Max num of topics

    Returns:
    -------
    best_model :
    best_coherence_values 
    n_topics : numbmber of topics
    """
    best_coherence_value = -np.inf
    best_model = None
    n_topics = None
    for num_topics in range(start, end + 1, step):
        #print('\nTraining with n_topics = {}, training sample = {}.'.format(num_topics,len(corpus)))
        np.random.seed(seed=1)
        model = gensim.models.wrappers.LdaMallet(mallet_path=mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary, 
                                                 alpha=alpha, optimize_interval=10, iterations = 2000, random_seed=1)
                                                 #prefix=os.path.join(model_folder,"mallet_{}_topics_".format(num_topics)))
        model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(model)
        #print('Calculating coherence score based on {} samples.'.format(len(texts)))
        
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_value = coherencemodel.get_coherence()
        if coherence_value > best_coherence_value:
            best_coherence_value = coherence_value
            best_model = model
            n_topics = num_topics
        
        #print("\n{}: {}".format(num_topics,coherence_value))
        
    return best_model, best_coherence_value, n_topics

In [24]:
def fine_tune_lda_alpha(corpus, dictionary, texts, mallet_path, start_alpha=0.01, end_alpha=10, number=10, n_topics=10):
    """
    Compute c_v coherence for various number of topics
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts

    Returns:
    -------
    best_model
    best_coherence_values
    best_alpha 
    """
    best_coherence_value = -np.inf
    best_model = None
    best_alpha = None
    alphas = np.logspace(np.log10(start_alpha), np.log10(end_alpha), num=number)
    for alpha in alphas:
        np.random.seed(seed=1)
        model = gensim.models.wrappers.LdaMallet(mallet_path=mallet_path, corpus=corpus, num_topics=n_topics, id2word=dictionary, 
                                                 alpha=alpha, optimize_interval=10, iterations = 2000, random_seed=1)
                                            
        model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(model)
        
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_value = coherencemodel.get_coherence()
        if coherence_value > best_coherence_value:
            best_coherence_value = coherence_value
            best_model = model
            best_alpha = alpha
        
        #print("\n{}: {}".format(num_topics,coherence_value))
        
    return best_model, best_coherence_value, best_alpha

In [25]:
res = []
min_topics = 10
max_topics = 50
step = 1
tmp_folder = '/tmp'
num_topics = range(min_topics, max_topics + 1, step)
for n_topics in num_topics:
    model, coherence_value, alpha = fine_tune_lda_alpha(RAM_corpus, lda_dict, lemma_corpus, mallet_path,\
                                                        start_alpha=0.01, end_alpha=10, number=10,\
                                                        n_topics=n_topics)
    print("\n{} topics (best alpha: {:.2f}): {:.4f}".format(n_topics, alpha, coherence_value))
    res.append((model, coherence_value, alpha, n_topics))
    #model.save(os.path.join(model_folder,'mallet_weights_{}_{:.2f}'.format(n_topics, alpha).replace('.', '_')))
    #vis_data = gensim_vis.prepare(model,corpus=RAM_corpus, dictionary=lda_dict)
    #pyLDAvis.save_html(vis_data, os.path.join(result_folder,'mallet_{}_topics_short_run.html'.format(n_topics))) 
    if len([name for name in os.listdir(tmp_folder) if os.path.isfile(os.path.join(tmp_folder, name))]) > 5000:
        [os.remove(os.path.join(tmp_folder,f)) for f in os.listdir(tmp_folder) if f.endswith(".mallet.gz") \
                                                                                  or f.endswith(".mallet") \
                                                                                  or f.endswith(".txt")]

INFO:serializing temporary corpus to /tmp/1e4cf9_corpus.txt
INFO:converting temporary corpus to MALLET format with /mnt/packages/Mallet/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/1e4cf9_corpus.txt --output /tmp/1e4cf9_corpus.mallet


CalledProcessError: Command '/mnt/packages/Mallet/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/1e4cf9_corpus.txt --output /tmp/1e4cf9_corpus.mallet' returned non-zero exit status 126.

In [26]:
best_model = None
best_score = -np.inf
best_num_topics = None
n_topics = 10
to_print = []
#print("Number of Alphas is {}".format(len(res)))
for result in res:
    if result[1] > best_score:
        best_model = result[0]
        best_score = result[1]
        best_num_topics = n_topics
    to_print.append("model with {0} topics has a coherence score of {1:.4f}".format(n_topics, result[1]))
    n_topics += 1
    
with open(os.path.join(result_folder, 'coherence_score.txt'), 'w', encoding='utf8') as f:
    f.writelines('\n'.join(to_print))

In [27]:
model = gensim.models.wrappers.LdaMallet(mallet_path=mallet_path, corpus=RAM_corpus, num_topics=37, id2word=lda_dict, 
                                                 alpha=0.46, optimize_interval=10, iterations = 2000, random_seed=1,
                                                 prefix=os.path.join(model_folder,"mallet_{}_topics_".format(37)))
best_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(model)

INFO:serializing temporary corpus to /mnt/models/mallet_37_topics_corpus.txt
INFO:converting temporary corpus to MALLET format with /mnt/packages/Mallet/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /mnt/models/mallet_37_topics_corpus.txt --output /mnt/models/mallet_37_topics_corpus.mallet


CalledProcessError: Command '/mnt/packages/Mallet/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /mnt/models/mallet_37_topics_corpus.txt --output /mnt/models/mallet_37_topics_corpus.mallet' returned non-zero exit status 126.

In [29]:
load_model = False
model_file = 'best_model_weights_37_2019_03_21'
if load_model:
    best_model = gensim.models.wrappers.LdaMallet.load(os.path.join(model_folder,model_file))
    np.random.seed(seed=1)
    model = gensim.models.wrappers.LdaMallet(mallet_path=mallet_path, corpus=RAM_corpus, num_topics=best_model.num_topics, 
                                             id2word=lda_dict, alpha=1, optimize_interval=10, iterations = 2000, random_seed=1)
                                            
    model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(model)
    coherencemodel = CoherenceModel(model=model, texts=lemma_corpus, dictionary=lda_dict, coherence='c_v')
    coherence_value = coherencemodel.get_coherence()
    print("Coherence Value is: {}".format(coherence_value))
    pyLDAvis.enable_notebook()
    vis_data = gensim_vis.prepare(model,corpus=RAM_corpus, dictionary=lda_dict)
    vis_data

In [60]:
pyLDAvis.enable_notebook()
vis_data = gensim_vis.prepare(best_model,corpus=RAM_corpus, dictionary=lda_dict)
vis_data

/usr/local/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/usr/local/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/usr/local/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/usr/local/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:208: RuntimeWarning: invalid value encountered in multiply
  relevance = lambda_ * log_ttd + (1 - lambda_) * log_lift
/usr/local/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:208: RuntimeWarning: invalid value encountered in multiply
  relevance = lambda_ * log_ttd + (1 - lambda_) * log_lift


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
2      8.539077        1       1 -0.078778 -0.008033
5      5.750911        1       2 -0.106805  0.135881
1      5.450325        1       3 -0.043510  0.049115
18     4.541671        1       4 -0.028891 -0.030842
0      4.376081        1       5 -0.117159  0.169321
3      4.174292        1       6 -0.119801  0.125716
43     3.819800        1       7 -0.074674  0.038760
17     3.638919        1       8 -0.075117  0.021776
33     3.334632        1       9 -0.118598  0.130432
24     3.267372        1      10 -0.102606  0.041446
44     3.059600        1      11 -0.075971 -0.094172
29     2.952043        1      12 -0.079148  0.113725
15     2.861327        1      13 -0.114141  0.144501
37     2.758153        1      14  0.322327  0.033477
20     2.675615        1      15 -0.007314 -0.202258
31     2.467631        1      16 -0.105141  0.087840
35     2.442663        1      17  0.272720  0.033900
30     2.207170        1      18  0.320027  0.088462
8      2.054742        1      19  0.082956  0.078625
26     1.735232        1      20 -0.082921 -0.050448
12     1.698304        1      21 -0.024321  0.034155
39     1.655935        1      22 -0.107156 -0.015909
27     1.630144        1      23 -0.031662 -0.139975
9      1.417316        1      24 -0.024468 -0.025178
28     1.385341        1      25 -0.035790 -0.147323
13     1.376292        1      26 -0.049503  0.097507
23     1.331427        1      27 -0.005449 -0.076237
42     1.323959        1      28  0.010347  0.008149
25     1.272123        1      29 -0.006257  0.043496
11     1.242382        1      30 -0.066870  0.044492
21     1.234868        1      31  0.156894 -0.056095
7      1.185579        1      32  0.164870  0.149767
14     1.080538        1      33 -0.005464  0.091896
4      1.080356        1      34  0.361938  0.074049
22     1.044494        1      35  0.020685 -0.029088
38     1.015442        1      36 -0.013726 -0.213024
10     0.922547        1      37  0.032544 -0.198408
41     0.918824        1      38 -0.021642  0.012225
16     0.865983        1      39  0.060012 -0.034404
19     0.865896        1      40 -0.038060 -0.179565
40     0.846240        1      41  0.050070  0.006526
36     0.799215        1      42 -0.041718 -0.007817
6      0.677442        1      43  0.043200 -0.211427
32     0.591834        1      44 -0.085189 -0.029154
34     0.430263        1      45 -0.010741 -0.105881, topic_info=     Category         Freq                   Term        Total  loglift  \
term                                                                      
1954  Default  1305.000000                foreign  1305.000000  30.0000   
1661  Default  1045.000000               exchange  1045.000000  29.0000   
1960  Default  1017.000000       foreign_exchange  1017.000000  28.0000   
2162  Default  3646.000000                 growth  3646.000000  27.0000   
3315  Default   818.000000                    oil   818.000000  26.0000   
3601  Default  1901.000000                 policy  1901.000000  25.0000   
1881  Default  2848.000000                 fiscal  2848.000000  24.0000   
3924  Default  1283.000000                   rate  1283.000000  23.0000   
1846  Default  2383.000000              financial  2383.000000  22.0000   
370   Default  1678.000000                   bank  1678.000000  21.0000   
3714  Default  1739.000000                  price  1739.000000  20.0000   
1968  Default   721.000000  foreign_exchange_rate   721.000000  19.0000   
1668  Default   728.000000          exchange_rate   728.000000  18.0000   
4044  Default  1604.000000                 reform  1604.000000  17.0000   
4400  Default  1362.000000                 sector  1362.000000  16.0000   
1080  Default   885.000000                   debt   885.000000  15.0000   
4274  Default  1881.000000                   risk  1881.000000  14.0000   
2825  Default  1846.000000               

In [61]:
n_words = 20
id_map = dict(zip(np.array(vis_data.topic_order)- 1  , list(range(1,n_topics+1))))
import datetime
date = datetime.datetime.now()
date = date.strftime("%Y_%m_%d")
df_d, df_t = save_results_to_excel(date, best_model, lda_dict, dataset_processed_folder, dataset_folder, best_num_topics, n_words, id_map, result_folder)

In [62]:
pyLDAvis.save_html(vis_data,  os.path.join(result_folder,'best_model_{}_topics_{}.html'.format(best_num_topics, date))) 

In [ ]:
best_model.save(os.path.join(model_folder,'best_model_weights_{}_{}'.format(best_num_topics, date)))

In [13]:
df_d.iloc[4]['text']

'Protracted economic and financial volatility, especially for emerging markets  (triggered by prospective exit from UMP).\nProspects of higher interest rates in advanced economies could trigger a sustained reversal of capital flows from the region, intensifying foreign currency liquidity strains.\nPressure on international reserves and the exchange rate, with effects on balance sheets, and potentially, a reacceleration of dollarization.\nAllow the currency to depreciate and refrain from domestic demand stimulus to preserve limited international reserves and contain inflation.\nLower than anticipated emerging market growth potential  (earlier maturing of the cycle and incomplete structural reforms with spillovers to LICs and advanced economies).\nGrowth relies to a large extent on trade and FDI links with other Asian countries, particularly China, Thailand and Vietnam.\nSlowdown in trading partner growth will put pressure on the balance of payments as exports decline and domestic demand

In [17]:
n_topics = 22
n_words = 20
#np.random.seed(seed=1)
lda_mallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=RAM_corpus, num_topics=n_topics,
                                              id2word=lda_dict, alpha=0.02, optimize_interval=10, 
                                              iterations = 2000,)
                                              #prefix=os.path.join(model_folder,"mallet_{}_topics_".format(n_topics)))
lda_gensim = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_mallet)

INFO:serializing temporary corpus to /tmp/424e7f_corpus.txt
INFO:converting temporary corpus to MALLET format with /mnt/packages/Mallet/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/424e7f_corpus.txt --output /tmp/424e7f_corpus.mallet
INFO:training MALLET LDA with /mnt/packages/Mallet/bin/mallet train-topics --input /tmp/424e7f_corpus.mallet --num-topics 22  --alpha 0.02 --optimize-interval 10 --num-threads 4 --output-state /tmp/424e7f_state.mallet.gz --output-doc-topics /tmp/424e7f_doctopics.txt --output-topic-keys /tmp/424e7f_topickeys.txt --num-iterations 2000 --inferencer-filename /tmp/424e7f_inferencer.mallet --doc-topics-threshold 0.0  --random-seed 0
INFO:loading assigned topics from /tmp/424e7f_state.mallet.gz
INFO:using serial LDA version on this node


In [16]:
print('calculating coherence socre for {} documents ......'.format(len(lemma_corpus)))
coherence_model_lda = CoherenceModel(model=lda_gensim, texts=lemma_corpus, dictionary=lda_dict, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

INFO:using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate probabilities from sliding windows


calculating coherence socre for 669 documents ......


INFO:1 batches submitted to accumulate stats from 64 documents (11693 virtual)
INFO:2 batches submitted to accumulate stats from 128 documents (22462 virtual)
INFO:3 batches submitted to accumulate stats from 192 documents (34558 virtual)
INFO:4 batches submitted to accumulate stats from 256 documents (45410 virtual)
INFO:5 batches submitted to accumulate stats from 320 documents (57965 virtual)
INFO:6 batches submitted to accumulate stats from 384 documents (69956 virtual)
INFO:7 batches submitted to accumulate stats from 448 documents (80844 virtual)
INFO:8 batches submitted to accumulate stats from 512 documents (93064 virtual)
INFO:9 batches submitted to accumulate stats from 576 documents (106047 virtual)
INFO:10 batches submitted to accumulate stats from 640 documents (119000 virtual)
INFO:11 batches submitted to accumulate stats from 704 documents (124094 virtual)
INFO:serializing accumulator to return to master...
INFO:accumulator serialized
INFO:serializing accumulator to retu


Coherence Score:  0.3587213739024996


In [22]:
model, coherence_value, n = fine_tune_lda(RAM_corpus, lda_dict, lemma_corpus, mallet_path, alpha=0.02)

NameError: name 'fine_tune_lda' is not defined